# Notebook traduit en français d'après celui du cours "Deep Learning" d'Alfredo Canziani de la New York University  : 
https://github.com/Atcold/pytorch-Deep-Learning/blob/master/03-autograd_tutorial.ipynb

# Autograd : différenciation automatique

Le package ``autograd`` fournit une différenciation automatique pour toutes les opérations sur les tenseurs. Il s'agit d'un cadre défini par l'utilisateur, ce qui veut dire que votre toile de fond est
défini par la façon dont votre code est exécuté, et que chaque itération peut être différent.


In [1]:
import torch

Créer un tenseur :

In [2]:
# Créer un tenseur 2x2 avec des capacités d'accumulation de gradients
x = torch.tensor([[1, 2], [3, 4]], requires_grad=True, dtype=torch.float32)
print(x)

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)


Effectuer une opération sur le tenseur :

In [3]:
# Déduire 2 de tous les éléments
y = x - 2
print(y)

tensor([[-1.,  0.],
        [ 1.,  2.]], grad_fn=<SubBackward0>)


``y`` a été créé à la suite d'une opération, il a donc un ``grad_fn``

In [4]:
print(y.grad_fn)

In [ ]:
# Qu'est-ce qui se passe ici ?
print(x.grad_fn)

In [6]:
# Creusons un peu plus loin...
y.grad_fn

In [7]:
y.grad_fn.next_functions[0][0]

In [8]:
y.grad_fn.next_functions[0][0].variable

tensor([[1., 2.],
        [3., 4.]], requires_grad=True)

In [9]:
# Faire plus d'opérations sur  y
z = y * y * 3
a = z.mean()  # moyenne

print(z)
print(a)

tensor([[ 3.,  0.],
        [ 3., 12.]], grad_fn=<MulBackward0>)
tensor(4.5000, grad_fn=<MeanBackward0>)


In [10]:
# # Visualisons le graphique de calcul ! (Afredo tient à remercier ici @szagoruyko)
from torchviz import make_dot

In [21]:
make_dot(a)

## Gradients

Rétropropagons maintenant `out.backward()`. Cela équivaut à faire `out.backward(torch.tensor([1.0]))`.

In [12]:
# Rétropropagation
a.backward()

Affichage des gradients $\frac{\text{d}a}{\text{d}x}$.




In [13]:
# Calculez-le à la main AVANT de l'exécuter
print(x.grad)

tensor([[-1.5000,  0.0000],
        [ 1.5000,  3.0000]])


Vous pouvez faire beaucoup de choses avec autograd !
> Avec une grande *flexibilité* vient une grande responsabilité

In [14]:
# Graphes dynamiques!
x = torch.randn(3, requires_grad=True)

y = x * 2
i = 0
while y.data.norm() < 1000:
    y = y * 2
    i += 1
print(y)

tensor([  12.5663, -995.8140,  188.6319], grad_fn=<MulBackward0>)


In [15]:
# # Si nous ne faisons pas de retour en arrière sur un scalaire, nous devons spécifier le grad_output
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


In [16]:
# AVANT d'éxécuter la cellule, pouvez-vous dire ce qui sera affiché ?
print(i)

8


## Inférence

In [17]:
# Cette variable détermine l'étendue du tenseur en dessous
n = 3

In [18]:
# x et w permettent l'accumulation de gradients
x = torch.arange(1., n + 1, requires_grad=True)
w = torch.ones(n, requires_grad=True)
z = w @ x
z.backward()
print(x.grad, w.grad, sep='\n')

tensor([1., 1., 1.])
tensor([1., 2., 3.])


In [19]:
# Seulement w permet l'accumulation de gradient
x = torch.arange(1., n + 1)
w = torch.ones(n, requires_grad=True)
z = w @ x
z.backward()
print(x.grad, w.grad, sep='\n')

None
tensor([1., 2., 3.])


In [20]:
x = torch.arange(1., n + 1)
w = torch.ones(n, requires_grad=True)

# Indépendamment de ce que vous faites dans ce contexte, tous les tenseurs de torch n'auront pas d'accumulation de gradient
with torch.no_grad():
    z = w @ x

try:
    z.backward()  # PyTorch va lancer une erreur ici, puisque z n'a pas de grad accum.
except RuntimeError as e:
    print('RuntimeError!!! >:[')
    print(e)

RuntimeError!!! >:[
element 0 of tensors does not require grad and does not have a grad_fn


## Plus de choses

La documentation relative au paquet de différenciation automatique se trouve à l'adresse suivante
http://pytorch.org/docs/autograd.